In [119]:
import re
import depfinder
import yaml
import _sre

In [120]:
config = yaml.load(open('conda-skeletor.yml', 'r'))

In [125]:
test_regexers = [re.compile(reg) for reg in config['test_regex']]
ignore_path_regexers = [re.compile(reg) for reg in config['ignore_path_regex']]
include_path_regexers = [re.compile(reg) for reg in config['include_path_regex']]
extra_setup_files_regexers = [re.compile(reg) for reg in config['extra_setup_files_regex']]

In [126]:
skxray_deps = list(depfinder.iterate_over_library('/home/edill/dev/python/scikit-xray'))

...................................................................................

In [129]:
skxray_deps[2]

('setupext',
 '/home/edill/dev/python/scikit-xray/setupext.py',
 ImportCatcher: {'builtin': {'sys', 'distutils', 'copy', 'os'}, 'required': {'numpy', 'six'}})

In [89]:
def _regex_tester(test_string, regexers):
    """Helper function to test a string against an iterable of regexers"""
    matched = False
    for regex in regexers:
        if regex.__class__.__name__ == 'SRE_Pattern':
            matched = regex.match(test_string) is not None
        else:
            matched = regex in test_string
        if matched:
            break
    return matched

In [92]:
def split_deps(iterable_of_deps_tuples, iterable_of_regexers):
    good = []
    bad = []
    for mod, mod_path, catcher in iterable_of_deps_tuples:
        # try the module name first
        mod_match = _regex_tester(mod, iterable_of_regexers)
        if not mod_match:
            # if the module name did not match, try the full path
            mod_match = _regex_tester(mod_path, iterable_of_regexers)
        if mod_match:
            good.append((mod, mod_path, catcher))
            mod_match = False
        else:
            bad.append((mod, mod_path, catcher))
    
    return good, bad

In [114]:
ignored, without_ignored = split_deps(skxray_deps, ignore_path_regexers)
included, without_included = split_deps(without_ignored, include_path_regexers)
tests, without_tests = split_deps(included, test_regexers)

In [116]:
for mod, path, catcher in without_tests:
    print(path)

/home/edill/dev/python/scikit-xray/skxray/fluorescence.py
/home/edill/dev/python/scikit-xray/skxray/diffraction.py
/home/edill/dev/python/scikit-xray/skxray/_version.py
/home/edill/dev/python/scikit-xray/skxray/__init__.py
/home/edill/dev/python/scikit-xray/skxray/io/save_powder_output.py
/home/edill/dev/python/scikit-xray/skxray/io/binary.py
/home/edill/dev/python/scikit-xray/skxray/io/net_cdf_io.py
/home/edill/dev/python/scikit-xray/skxray/io/avizo_io.py
/home/edill/dev/python/scikit-xray/skxray/io/gsas_file_reader.py
/home/edill/dev/python/scikit-xray/skxray/io/__init__.py
/home/edill/dev/python/scikit-xray/skxray/core/dpc.py
/home/edill/dev/python/scikit-xray/skxray/core/recip.py
/home/edill/dev/python/scikit-xray/skxray/core/speckle.py
/home/edill/dev/python/scikit-xray/skxray/core/stats.py
/home/edill/dev/python/scikit-xray/skxray/core/spectroscopy.py
/home/edill/dev/python/scikit-xray/skxray/core/arithmetic.py
/home/edill/dev/python/scikit-xray/skxray/core/correlation.py
/home/e

In [117]:
# print found test modules
for mod, path, catcher in tests:
    print(path)

/home/edill/dev/python/scikit-xray/skxray/io/tests/test_api.py
/home/edill/dev/python/scikit-xray/skxray/io/tests/test_netCDF_io.py
/home/edill/dev/python/scikit-xray/skxray/io/tests/test_powder_output.py
/home/edill/dev/python/scikit-xray/skxray/io/tests/__init__.py
/home/edill/dev/python/scikit-xray/skxray/core/constants/tests/test_api.py
/home/edill/dev/python/scikit-xray/skxray/core/constants/tests/test_xrs.py
/home/edill/dev/python/scikit-xray/skxray/core/constants/tests/test_basic.py
/home/edill/dev/python/scikit-xray/skxray/core/constants/tests/test_xrf.py
/home/edill/dev/python/scikit-xray/skxray/core/constants/tests/__init__.py
/home/edill/dev/python/scikit-xray/skxray/core/tests/test_feature.py
/home/edill/dev/python/scikit-xray/skxray/core/tests/test_roi.py
/home/edill/dev/python/scikit-xray/skxray/core/tests/test_recip.py
/home/edill/dev/python/scikit-xray/skxray/core/tests/test_image.py
/home/edill/dev/python/scikit-xray/skxray/core/tests/test_utils.py
/home/edill/dev/pyth

In [15]:
test_regex.match('py.test.py')

<_sre.SRE_Match object; span=(0, 10), match='py.test.py'>